In [82]:
import pandas as pd, re

In [41]:
csvs = {
    "cluster": "Cluster_merge_raw.csv",
    "realtor": "Realtor - Market Hotness.csv",
    "statsAmerica": "StatsAmerica - Population by Age and Sex - Clean.csv"
}

In [105]:
# read in, clean FIPS data

df_cluster = pd.read_csv(csvs["cluster"])

df_unemp = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

df_cluster = (
    df_cluster[
        (df_cluster.ibrc_geo_id_pop == df_cluster.ibrc_geo_id_combo)
        ]
    .drop(["Unnamed: 0", "ibrc_geo_id_pop", "ibrc_geo_id_sc", "county_fips_hot", "description_sc"], axis=1)
    .rename(columns={
        "ibrc_geo_id_combo": "fips"
    })
)

df_cluster.fips = [
    str(int(x)).zfill(5) for x in df_cluster.fips
]

df_cluster = df_cluster.merge(df_unemp, on="fips")

In [106]:
# organize columns

index_cols = ["fips", "description_pop", "year_pop"]

big5_cols = ['Agreeableness_sc', 'Conscientiousness_sc', "Extraversion_sc", "Neuroticism_sc", "Openness_sc"]

demographic_cols = ["Religiosity_sc"] + [x for x in df_cluster.columns if "population" in x]

pol_cols = ['Belief In Science_sc', 'Collectivism_sc', "Gender Equality_sc"]

personality_cols = [
    'Conflict Awareness_sc', 'Empathy_sc', "Hopefulness_sc",
     "Risk Taking_sc", "Selflessness_sc", "Tolerance_sc",
     "Work Ethic_sc"
     ]

economy_cols = ["Employment Rate_sc", "unemp", "Entrepreneurship_sc", "Income Mobility_sc", "Income Per Capita_sc"]

hot_cols = [x for x in df_cluster.columns if x.endswith("_hot")]

In [107]:
# add proportion variables to population by age columns

demographic_prop_cols = []

for pop_col in [x for x in demographic_cols if x.startswith("population_")] + ["male_population_pop", "female_population_pop"]:
    df_cluster[f"prop_{pop_col}"] = df_cluster[pop_col]/df_cluster["total_population_pop"]

    demographic_prop_cols += [f"prop_{pop_col}"]

In [114]:
# normalize features

df_normalized = df_cluster.copy()

for col in df_normalized[[x for x in df_normalized.columns if x not in index_cols + hot_cols]]:

    # put to scale 1-100. this is an issue as it assumes a uniform distribution. percentiles not calculated accurately.
    # df_normalized[f"{col}_norm"] = (df_normalized[col] - df_normalized[col].min())*(100/(df_normalized[col].max() - df_normalized[col].min()))

    # calculated as z-score. assumes normal distribution.

    df_normalized[f"{col}_norm"] = (df_normalized[col] - df_normalized[col].mean()) / df_normalized[col].std()

In [117]:
# save data

df_normalized.to_csv("cluster_merge_clean_normalized.csv", index=None)

---

In [198]:
df0 = pd.read_csv("cluster_merge_clean_normalized.csv", dtype={"fips": str})

assume median household has 3 bedrooms<br>
assume rent is 0.8% of housing price<br>
assume rent is 20% more than the mortgage<br>

In [237]:
df_preds0 = pd.read_csv("price_predictions_new.csv", dtype={"county_fips": str}).rename(columns={"county_fips": "fips"})
df_clust0 = pd.read_csv("fip_clusters.csv", dtype={"fips": int}).drop("Unnamed: 0", axis=1)
df_other0 = pd.read_csv("Cluster_merge_clean.csv", dtype={"fips_id": int}).drop("Unnamed: 0", axis=1).rename(columns={"fips_id": "fips"})

In [244]:
# add rent, mortgage payment

df_preds = df_preds0.copy()

df_preds.fips = [
    str(int(x)).zfill(5) for x in df_preds.fips
]

# df_preds.predicted_price = [
#     float(x.replace("[", '').replace("]", ""))
#     for x in df_preds.predicted_price
# ]

# df_preds = df_preds.groupby("fips").agg({"predicted_price": "mean"}).reset_index()

df_preds["median_rent"] = df_preds.predicted_price*0.008/3
df_preds["median_mortgage"] = df_preds.median_rent*3/1.2

# df_preds = df_preds.rename(columns={
#     "predicted_price": "Forecasted Median Housing Price"
# })

df_preds["predicted_price"] = [
    f"${round(x/1000)}k"
    for x in df_preds["predicted_price"]
]


In [245]:
# add clusters, cluster labels

cluster_label_dict = {
    0: "Best of Both Worlds",
    1: "Country Roads",
    2: "Star Gazing",
    3: "Big City Life",
    4: "American Dream"
}

df_clust = df_clust0.copy()

df_clust.fips = [
    str(int(x)).zfill(5) for x in df_clust.fips
]

df_clust["cluster_label"] = [
    cluster_label_dict[x]
    for x in df_clust.cluster
]

df_clust = df_clust[[
    "fips", "cluster", "cluster_label"
]]

In [246]:
# get education, unemployment data

df_other = df_other0.copy()

df_other.fips = [
    str(int(x)).zfill(5) for x in df_other.fips
]

other_cols = ['percent_of_adults_with_less_than_a_high_school_diploma_2015-19_edu',
 'percent_of_adults_with_a_high_school_diploma_only_2015-19_edu',
 'percent_of_adults_completing_some_college_or_associates_degree_2015-19_edu',
 'percent_of_adults_with_a_bachelors_degree_or_higher_2015-19_edu',
 'unemployment_rate_2020_emp']

df_other = df_other[['fips'] + other_cols]



In [247]:
# merge everything, define educated col, normalize educated and unemployment cols

df = df0.copy()

df = (
    df.merge(
        df_preds, on="fips", how="left"        
    )
    .merge(
        df_clust, on="fips", how="left"
    )
    .merge(
        df_other, on="fips", how="left"
    )
)

df.predicted_price = df.predicted_price.fillna("Not Available")
df.median_mortgage = df.median_mortgage.fillna(0)
df.median_rent = df.median_rent.fillna(0)

df = df.rename(columns={
    'percent_of_adults_with_less_than_a_high_school_diploma_2015-19_edu': "edu_no_hs",
    'percent_of_adults_with_a_high_school_diploma_only_2015-19_edu': "edu_hs",
    'percent_of_adults_completing_some_college_or_associates_degree_2015-19_edu': "edu_some_uni",
    'percent_of_adults_with_a_bachelors_degree_or_higher_2015-19_edu': "edu_uni",
    "unemployment_rate_2020_emp": "unemployment"
})

df["educated"] = 2*df["edu_uni"] + df["edu_some_uni"] - df["edu_hs"] - 2*df["edu_no_hs"]

df["employed"] = 100 - df.unemployment

for col in ["educated", "employed"]:

    df[f"{col}_norm"] = (df[col] - df[col].mean())/df[col].std()

# modify a couple other columns

df["left_wing_norm"] = df["Collectivism_sc_norm"]
df["right_wing_norm"] = df["Collectivism_sc_norm"]*-1

In [248]:
df.to_csv("county_data_final.csv")

---
sandbox

In [85]:
val = "Mortgage Budget"
str.lower(val[:re.search("Budget", val).start()-1])

'mortgage'

In [234]:
df_preds

,fips,predicted_price,median_rent,median_mortgage
0,01001,$239k,638.480000,1596.200000
1,01003,$410k,1093.179493,2732.948733
2,01009,$226k,601.939573,1504.848933
3,01015,$161k,428.396867,1070.992167
4,01017,$131k,348.378973,870.947433
...,...,...,...,...
1573,56025,$255k,679.495160,1698.737900
1574,56029,$539k,1437.626000,3594.065000
1575,56033,$366k,977.302373,2443.255933
1576,56037,$231k,616.044280,1540.110700
